In [1]:
import logging
import pandas as pd
import datetime
import os
os.chdir("../")

In [2]:
logging.basicConfig(filename="dataset/logs.log", level=logging.INFO)

In [3]:
logger = logging.getLogger()

In [4]:
data = pd.read_csv("dataset/house_price_prep.csv")
configs = pd.read_excel("assets/metadado.xlsx", sheet_name="quality")

In [5]:
data.head()

,id,preco,quartos,banheiros,area_util,area_total,andares,vista_mar,qualidade_vista
0,7229300521,231300.0,2,1,1180.0,5650,1,0,0
1,6414100192,538000.0,3,2,2570.0,7242,2,0,0
2,5631500400,180000.0,2,1,770.0,10000,1,0,0
3,2487200875,604000.0,4,3,1960.0,5000,1,0,0
4,1954400510,510000.0,3,2,1680.0,8080,1,0,0


In [6]:
configs

,variavel,key_validation,limits_validation,null_validation,operator,value,value_dev
0,id,1.0,NaN,NaN,NaN,NaN,NaN
1,quartos,NaN,1.0,NaN,<,5.0,NaN
2,area_total,NaN,NaN,1.0,NaN,0.1,0.05


In [7]:

i_limits = configs.loc[configs['limits_validation'] == 1].index
keys = list(configs.loc[configs['key_validation'] == 1]["variavel"])
i_null = configs.loc[configs['null_validation'] == 1].index

In [8]:
if data[keys].nunique().item() != len(data):
            print(keys)
            logger.warning(f"KEY_DUPLICATION ; {keys}; {datetime.datetime.now()}")


['id']


In [10]:
data[data["id"] == 795000620]

,id,preco,quartos,banheiros,area_util,area_total,andares,vista_mar,qualidade_vista
17602,795000620,115000.0,3,1,1080.0,6250,1,0,0
17603,795000620,124000.0,3,1,1080.0,6250,1,0,0
17604,795000620,157000.0,3,1,1080.0,6250,1,0,0


In [11]:
i_null

Index([2], dtype='int64')

In [12]:
configs.iloc[1]["value"]

5.0

In [13]:
for i in i_null:
            value = configs.iloc[i]["value"]
            col = configs.iloc[i]["variavel"]
            value_dev = configs.iloc[i]["value_dev"]
            n_null = len(data.loc[data[col].isnull()])/len(data)
            print(col, n_null)
            if not value - value_dev <= n_null <= value + value_dev:
                logger.warning(f"NULL_TOLERANCE ; {col}; {datetime.datetime.now()}")

area_total 0.0


In [14]:
for i in i_limits:
    value = configs.iloc[i]["value"]
    operator = configs.iloc[i]["operator"]
    col = configs.iloc[i]["variavel"]
    execution = f"data.loc[data.{col} {operator} {value}]"
    print(execution)
    n =len(eval(execution))
    if n > 0:
        logger.warning(f"{n} CASES OUT OF BOUNDS ; {col}; {datetime.datetime.now()}")

data.loc[data.quartos < 5.0]
